### Para o PPO a estrutura é muito parecida

In [1]:
import gymnasium as gym
import numpy as np
import tensorboard

%load_ext tensorboard

import stable_baselines3
stable_baselines3.__version__

'2.1.0'

In [2]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# para treinar com ambientes Atari
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack

In [3]:
from stable_baselines3.ppo import CnnPolicy

In [4]:
# Defina os hiperparâmetros desejados
gamma = 0.99
learning_rate = 0.0003
n_epochs = 10
n_steps = 2046

ENV_NAME = 'ALE/SpaceInvaders-v5'
env = make_atari_env(ENV_NAME, n_envs=4, seed=12306488)
env = VecFrameStack(env, n_stack=4)

# Cria com uma rede convolucional como política e define os hiperparâmetros
model = PPO(CnnPolicy, env, gamma=gamma, learning_rate=learning_rate, n_epochs=n_epochs, tensorboard_log="log_dir", n_steps=n_steps, verbose=1)


Using cuda device
Wrapping the env in a VecTransposeImage.


c:\Users\Pichau\anaconda3\envs\rl23\Lib\site-packages\stable_baselines3\ppo\ppo.py:148: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 8184`, after every 127 untruncated mini-batches, there will be a truncated mini-batch of size 56
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2046 and n_envs=4)
  warnings.warn(


In [13]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6006 (pid 18444), started 1 day, 17:01:50 ago. (Use '!kill 18444' to kill it.)

In [32]:
# Treina por 200 mil passos -- pode demorar mais de 5 min!
model.learn(total_timesteps=100_000)

Logging to log_dir\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 550      |
|    ep_rew_mean     | 166      |
| time/              |          |
|    fps             | 231      |
|    iterations      | 1        |
|    time_elapsed    | 35       |
|    total_timesteps | 8184     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 494         |
|    ep_rew_mean          | 129         |
| time/                   |             |
|    fps                  | 191         |
|    iterations           | 2           |
|    time_elapsed         | 85          |
|    total_timesteps      | 16368       |
| train/                  |             |
|    approx_kl            | 0.009032685 |
|    clip_fraction        | 0.101       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.78       |
|    explained_variance   | -0.00326    |
|    

### Para esse caso precisamos avaliar os valores utilizando a lib optuna

In [33]:
import optuna

Como sugerido vou manter o range de otimização das variáveis entre 50% e 150% do valor original 

In [34]:
def train_PPO(trial : optuna.Trial):
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    n_steps    = trial.suggest_int('n_steps', 1023, 3069)
    gamma      = trial.suggest_float('gamma', 0.5, 1.50)
    lr         = trial.suggest_float('lr', 0.00015, 0.00045)
    n_epochs   = trial.suggest_int('n_epochs', 5, 15)
    # Defina os hiperparâmetros desejado

    ENV_NAME = 'ALE/SpaceInvaders-v5'
    env = make_atari_env(ENV_NAME, n_envs=4, seed=12306488)
    env = VecFrameStack(env, n_stack=4)

    # Crie o modelo com os hiperparâmetros sugeridos pelo Optuna
    model = PPO(CnnPolicy, env, gamma=gamma, learning_rate=lr, n_epochs=n_epochs, n_steps=n_steps, verbose=1)

    print(f"\nTRIAL NUMBER #{trial.number}: {trial.params}")

    # Treine o modelo por um número fixo de etapas
    model.learn(total_timesteps=100_000)

    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=10)

    return mean_reward

In [35]:
study_PPO = optuna.create_study(direction='maximize',
                        storage='sqlite:///stud_PPO2.db',
                        study_name='validation_PPO',
                        load_if_exists=True)

# maximiza o valor de retorno de train_exp_sarsa, rodando "n_trials" vezes
# o parâmetro "n_jobs" indica a quantidade de CPUs a serem usadas (-1 para usar todas)
study_PPO.optimize(train_PPO, n_trials=12, n_jobs=-1)


[I 2024-02-18 05:48:15,673] A new study created in RDB with name: validation_PPO


Using cuda device
Wrapping the env in a VecTransposeImage.


c:\Users\Pichau\anaconda3\envs\rl23\Lib\site-packages\stable_baselines3\ppo\ppo.py:148: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 9688`, after every 151 untruncated mini-batches, there will be a truncated mini-batch of size 24
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2422 and n_envs=4)
  warnings.warn(


Using cuda device
Wrapping the env in a VecTransposeImage.

TRIAL NUMBER #2: {'n_steps': 2422, 'gamma': 0.850299704721077, 'lr': 0.0002281152050716582, 'n_epochs': 6}


c:\Users\Pichau\anaconda3\envs\rl23\Lib\site-packages\stable_baselines3\ppo\ppo.py:148: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 9592`, after every 149 untruncated mini-batches, there will be a truncated mini-batch of size 56
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2398 and n_envs=4)
  warnings.warn(


...

[I 2024-02-18 07:17:45,955] Trial 10 finished with value: 279.5 and parameters: {'n_steps': 2157, 'gamma': 0.5671624686016697, 'lr': 0.00037206379938648426, 'n_epochs': 9}. Best is trial 7 with value: 365.0.


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 560         |
|    ep_rew_mean          | 181         |
| time/                   |             |
|    fps                  | 32          |
|    iterations           | 5           |
|    time_elapsed         | 1850        |
|    total_timesteps      | 60140       |
| train/                  |             |
|    approx_kl            | 0.050047863 |
|    clip_fraction        | 0.449       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.69       |
|    explained_variance   | 0.248       |
|    learning_rate        | 0.000158    |
|    loss                 | -0.0882     |
|    n_updates            | 60          |
|    policy_gradient_loss | -0.0868     |
|    value_loss           | 0.0545      |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 604     

[I 2024-02-18 07:21:10,503] Trial 8 finished with value: 292.0 and parameters: {'n_steps': 2426, 'gamma': 0.7769360757451391, 'lr': 0.00019881786569299022, 'n_epochs': 13}. Best is trial 7 with value: 365.0.


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 592        |
|    ep_rew_mean          | 212        |
| time/                   |            |
|    fps                  | 35         |
|    iterations           | 6          |
|    time_elapsed         | 2006       |
|    total_timesteps      | 72168      |
| train/                  |            |
|    approx_kl            | 0.06316606 |
|    clip_fraction        | 0.501      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.65      |
|    explained_variance   | 0.276      |
|    learning_rate        | 0.000158   |
|    loss                 | -0.105     |
|    n_updates            | 75         |
|    policy_gradient_loss | -0.0933    |
|    value_loss           | 0.0522     |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 544        |
|    ep_rew_mean

[I 2024-02-18 07:26:13,218] Trial 11 finished with value: 205.0 and parameters: {'n_steps': 3007, 'gamma': 0.7598685207617438, 'lr': 0.0001577475259813369, 'n_epochs': 15}. Best is trial 7 with value: 365.0.


In [36]:
print("MELHORES PARÂMETROS:")
print(study_PPO.best_params)

MELHORES PARÂMETROS:
{'n_steps': 2287, 'gamma': 0.734780302742167, 'lr': 0.0003629340451380195, 'n_epochs': 11}


In [37]:
# Defina os hiperparâmetros desejados
gamma = study_PPO.best_params["gamma"]
learning_rate = study_PPO.best_params["lr"]
n_epochs = study_PPO.best_params["n_epochs"]
n_steps = study_PPO.best_params["n_steps"]

ENV_NAME = 'ALE/SpaceInvaders-v5'
env = make_atari_env(ENV_NAME, n_envs=4, seed=12306488)
env = VecFrameStack(env, n_stack=4)

# Cria com uma rede convolucional como política e define os hiperparâmetros
model = PPO(CnnPolicy, env, gamma=gamma, learning_rate=learning_rate, n_epochs=n_epochs, tensorboard_log="log_dir", n_steps=n_steps, verbose=1)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [14]:
%tensorboard --logdir=log_dir

Reusing TensorBoard on port 6009 (pid 20732), started 1 day, 9:43:22 ago. (Use '!kill 20732' to kill it.)

In [39]:
model.learn(total_timesteps=1_200_000)

Logging to log_dir_1\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 512      |
|    ep_rew_mean     | 128      |
| time/              |          |
|    fps             | 219      |
|    iterations      | 1        |
|    time_elapsed    | 41       |
|    total_timesteps | 9148     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 504        |
|    ep_rew_mean          | 140        |
| time/                   |            |
|    fps                  | 188        |
|    iterations           | 2          |
|    time_elapsed         | 97         |
|    total_timesteps      | 18296      |
| train/                  |            |
|    approx_kl            | 0.02612856 |
|    clip_fraction        | 0.196      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.78      |
|    explained_variance   | -0.0607    |
|    learning_rate

In [41]:
# Defina os hiperparâmetros desejados
gamma = study_PPO.best_params["gamma"]
learning_rate = study_PPO.best_params["lr"]
n_epochs = study_PPO.best_params["n_epochs"]
n_steps = study_PPO.best_params["n_steps"]

ENV_NAME = 'ALE/SpaceInvaders-v5'
env2 = make_atari_env(ENV_NAME, n_envs=4, seed=12306488)
env2 = VecFrameStack(env, n_stack=4)

# Cria com uma rede convolucional como política e define os hiperparâmetros
model2 = PPO(CnnPolicy, env2, gamma=gamma, learning_rate=learning_rate, n_epochs=n_epochs, tensorboard_log="log_dir", n_steps=n_steps, verbose=1)

Using cuda device
Wrapping the env in a VecTransposeImage.


c:\Users\Pichau\anaconda3\envs\rl23\Lib\site-packages\stable_baselines3\ppo\ppo.py:148: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 9148`, after every 142 untruncated mini-batches, there will be a truncated mini-batch of size 60
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2287 and n_envs=4)
  warnings.warn(


In [42]:
model2.learn(total_timesteps=5_000_000)

Logging to log_dir\PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 512      |
|    ep_rew_mean     | 124      |
| time/              |          |
|    fps             | 224      |
|    iterations      | 1        |
|    time_elapsed    | 40       |
|    total_timesteps | 9148     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 545         |
|    ep_rew_mean          | 143         |
| time/                   |             |
|    fps                  | 160         |
|    iterations           | 2           |
|    time_elapsed         | 114         |
|    total_timesteps      | 18296       |
| train/                  |             |
|    approx_kl            | 0.025526892 |
|    clip_fraction        | 0.186       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.78       |
|    explained_variance   | -0.0223     |
|    